In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
from datetime import datetime, timedelta
from typing import Dict, List  
import warnings

import openai
import json
from typing import Dict, List
import time
from tqdm import tqdm
import os
import re 
import signal 
import sys


In [ ]:
############### converting data to text for "no_cov_no_bmi" ###############

warnings.filterwarnings('ignore')

class Health_data_text_converter: 
    def __init__(self): 
        self.mappings = {
            'gender': {1: "male", 2: "female"}, 
            'race': {1: "Mexican American", 
                     2: "Other Hispanic", 
                     3: "Non-Hispanic White", 
                     4: "Non-Hispanic Black", 
                     5: "Other"},
            'education': {1: "less than 9th grade", 
                          2: "9-11th grade (includes 12th grade and no diploma)", 
                          3: "high school graduate/GED or equivalent", 
                          4: "some college or associates (AA) degree", 
                          5: "college graduate or higher"},
            'marital_status': {1: "married", 
                        2: "widowed", 
                        3: "divorced", 
                        4: "separated", 
                        5: "never married", 
                        6: "living with partner"},
            'physical_activity': {0: "sedentary", 
                                  1: "light activity",
                                  2: "lifestyle activity", 
                                  3: "moderate activity", 
                                  4: "vigorous activity"}
        }

    def convert_row_to_text(self, row: pd.Series) -> str: 
        covariates = {
            'seqn': row['seqn'],
            'age': row['age'],
            'gender': row['gender'],
            'race': row['race'],
            'education': row['education'],
            'marital_status': row['marital_status'],
            'pir': row['pir']
        }
        time_columns = [col for col in row.index if col.startswith('time')] 
        activity_data = [row[col] for col in time_columns if pd.notna(row[col])]
        return self.create_health_profile(covariates, activity_data)

    def create_health_profile(self, covariates: Dict, activity_data: List) -> str: 
        covariates_section = self.format_covariates(covariates)
        activity_section = self.analyze_activity(activity_data) 

        health_profile = f"""[Basic information]
{covariates_section}

[Physical Activity Assessment]
{activity_section}

[Health Context] 
Comprehensive health profile based on demographic characteristics and continuous physical activity monitoring using accelerometer data collected in 5-minute intervals.
"""
        return health_profile.strip()
 
    def format_covariates(self, var: Dict) -> str:      
        def safe_map(key, value):
            if pd.isna(value) or value not in self.mappings[key]:
                return "unknown"
            return self.mappings[key][value]
        
        pir_text = f"{var['pir']:.2f}" if pd.notna(var.get('pir')) else "unknown"
       
        return f"""Age: {var.get('age', 'unknown')} years old
Gender: {safe_map('gender', var.get('gender'))}
Race/Ethnicity: {safe_map('race', var.get('race'))}
Education: {safe_map('education', var.get('education'))}
Marital Status: {safe_map('marital_status', var.get('marital_status'))}
Poverty Income Ratio: {pir_text}"""
        
    def analyze_activity(self, activity_data: List) -> str:
        if not activity_data:                
            return "No activity data available"

        activity_array = np.array([x for x in activity_data if pd.notna(x) and 0 <= x <= 4])
        if len(activity_array) == 0:
            return "No valid activity data available"

        # activity distribution 
        activity_counts = pd.Series(activity_array).value_counts().sort_index()
        total_intervals = len(activity_array)
        
        # activity level percentage 
        percentages = {}
        for level in range(5):
            count = activity_counts.get(level, 0)
            percentages[level] = (count / total_intervals) * 100
        
        # dominant activity pattern
        dominant_level = activity_counts.idxmax()
        dominant_activity = self.mappings['physical_activity'][dominant_level]

        # active vs sedentary time
        active_intervals = sum(1 for x in activity_array if x >= 2)  # lifestyle, moderate, vigorous
        active_percentage = (active_intervals / total_intervals) * 100

        return f"""Monitoring Duration: {total_intervals} five-minute intervals recorded
Activity Distribution: {percentages[0]:.1f}% sedentary, {percentages[1]:.1f}% light, {percentages[2]:.1f}% lifestyle, {percentages[3]:.1f}% moderate, {percentages[4]:.1f}% vigorous
Primary Activity Pattern: predominantly {dominant_activity}
Active Time: {active_percentage:.1f}% of monitored time spent in active movement"""

    def batch_process(self, df: pd.DataFrame, output_file: str):
        """Process entire dataset"""
        results = []
        
        for idx, row in df.iterrows():
            try:
                text_profile = self.convert_row_to_text(row)
                results.append({
                    'seqn': int(row['seqn']),
                    'text_profile': text_profile,
                })
            except Exception as e:
                print(f"Error processing row {idx}: {e}")
        
        results_df = pd.DataFrame(results)
        results_df['seqn'] = results_df['seqn'].astype(int)
        
        results_df.to_csv(output_file, index=False)
        return results_df
        

In [ ]:
############### converting data to text for "cov_no_bmi" ###############

warnings.filterwarnings('ignore')

class Health_data_text_converter_cov: 

    def __init__(self): 
        self.mappings = {
            'gender': {1: "male", 2: "female"}, 
            'race': {1: "Mexican American", 
                     2: "Other Hispanic", 
                     3: "Non-Hispanic White", 
                     4: "Non-Hispanic Black", 
                     5: "Other"},
            'education': {1: "less than 9th grade", 
                          2: "9-11th grade (includes 12th grade and no diploma)", 
                          3: "high school graduate/GED or equivalent", 
                          4: "some college or associates (AA) degree", 
                          5: "college graduate or higher"},
            'marital_status': {1: "married", 
                        2: "widowed", 
                        3: "divorced", 
                        4: "separated", 
                        5: "never married", 
                        6: "living with partner"},
            'physical_activity': {0: "sedentary", 
                                  1: "light activity",
                                  2: "lifestyle activity", 
                                  3: "moderate activity", 
                                  4: "vigorous activity"}
        }

    def convert_row_to_text(self, row: pd.Series) -> str: 
        covariates = {
            'seqn': row['seqn'],
            'age': row['age'],
            'gender': row['gender'],
            'race': row['race'],
            'education': row['education'],
            'marital_status': row['marital_status'],
            'pir': row['pir'],
            'med_sbp': row['med_sbp'], 
            'med_dbp': row['med_dbp'], 
            'chol_total': row['chol_total'], 
            'chol_hdl': row['chol_hdl'], 
            'chol_ldl': row['chol_ldl'], 
            'triglyceride': row['triglyceride'], 
            'glucose': row['glucose'], 
            'crp': row['crp']
        }
        time_columns = [col for col in row.index if col.startswith('time')] 
        activity_data = [row[col] for col in time_columns if pd.notna(row[col])]
        return self.create_health_profile(covariates, activity_data)

    def create_health_profile(self, covariates: Dict, activity_data: List) -> str: 
        covariates_section = self.format_covariates(covariates)
        activity_section = self.analyze_activity(activity_data) 

        health_profile = f"""[Basic information]
{covariates_section}

[Physical Activity Assessment]
{activity_section}

[Health Context] 
Comprehensive health profile based on demographic characteristics and continuous physical activity monitoring using accelerometer data collected in 5-minute intervals.
"""
        return health_profile.strip()
 
    def format_covariates(self, var: Dict) -> str:      
        def safe_map(key, value):
            if pd.isna(value) or value not in self.mappings[key]:
                return "unknown"
            return self.mappings[key][value]
        
        pir_text = f"{var['pir']:.2f}" if pd.notna(var.get('pir')) else "unknown"
        med_sbp_text = f"{var['med_sbp']:.2f}" if pd.notna(var.get('med_sbp')) else "unknown"
        med_dbp_text = f"{var['med_dbp']:.2f}" if pd.notna(var.get('med_dbp')) else "unknown"
        chol_total_text = f"{var['chol_total']:.2f}" if pd.notna(var.get('chol_total')) else "unknown"
        chol_hdl_text = f"{var['chol_hdl']:.2f}" if pd.notna(var.get('chol_hdl')) else "unknown"
        chol_ldl_text = f"{var['chol_ldl']:.2f}" if pd.notna(var.get('chol_ldl')) else "unknown"
        triglyceride_text = f"{var['triglyceride']:.2f}" if pd.notna(var.get('triglyceride')) else "unknown"
        glucose_text = f"{var['glucose']:.2f}" if pd.notna(var.get('glucose')) else "unknown"
        crp_text = f"{var['crp']:.2f}" if pd.notna(var.get('crp')) else "unknown"
        
        return f"""Age: {var.get('age', 'unknown')} years old
Gender: {safe_map('gender', var.get('gender'))}
Race/Ethnicity: {safe_map('race', var.get('race'))}
Education: {safe_map('education', var.get('education'))}
Marital Status: {safe_map('marital_status', var.get('marital_status'))}
Poverty Income Ratio: {pir_text}
Median Systolic Blood Pressure: {med_sbp_text} mmHg
Median Diastolic Blood Pressure: {med_dbp_text} mmHg
Total Cholesterol: {chol_total_text} mg/dL
HDL: {chol_hdl_text} mg/dL
LDL: {chol_ldl_text} mg/dL
Triglyceride: {triglyceride_text} mg/dL
Glucose: {glucose_text} mg/dL
C-reactive Protein: {crp_text} mg/dL"""

        
    def analyze_activity(self, activity_data: List) -> str:
        if not activity_data:                
            return "No activity data available"

        activity_array = np.array([x for x in activity_data if pd.notna(x) and 0 <= x <= 4])
        if len(activity_array) == 0:
            return "No valid activity data available"

        # activity distribution 
        activity_counts = pd.Series(activity_array).value_counts().sort_index()
        total_intervals = len(activity_array)
        
        # activity level percentage 
        percentages = {}
        for level in range(5):
            count = activity_counts.get(level, 0)
            percentages[level] = (count / total_intervals) * 100
        
        # dominant activity pattern
        dominant_level = activity_counts.idxmax()
        dominant_activity = self.mappings['physical_activity'][dominant_level]

        # active vs sedentary time
        active_intervals = sum(1 for x in activity_array if x >= 2)  # lifestyle, moderate, vigorous
        active_percentage = (active_intervals / total_intervals) * 100

        return f"""Monitoring Duration: {total_intervals} five-minute intervals recorded
Activity Distribution: {percentages[0]:.1f}% sedentary, {percentages[1]:.1f}% light, {percentages[2]:.1f}% lifestyle, {percentages[3]:.1f}% moderate, {percentages[4]:.1f}% vigorous
Primary Activity Pattern: predominantly {dominant_activity}
Active Time: {active_percentage:.1f}% of monitored time spent in active movement"""


    def batch_process(self, df: pd.DataFrame, output_file: str):
        """Process entire dataset"""
        results = []
        
        for idx, row in df.iterrows():
            try:
                text_profile = self.convert_row_to_text(row)
                results.append({
                    'seqn': int(row['seqn']),
                    'text_profile': text_profile,
                })
            except Exception as e:
                print(f"Error processing row {idx}: {e}")
        
        results_df = pd.DataFrame(results)
        results_df['seqn'] = results_df['seqn'].astype(int)
        
        results_df.to_csv(output_file, index=False)
        return results_df
        

In [ ]:
############### Prompt WITHOUT baseline ###############
############################################################

def create_prompt(health_profile_text):
    prompt_template = """Your Role: You are a well-versed scholar in the study of health outcomes, and you are able to combine various health indicators to determine a person's overweight status, defined by Body Mass Index (BMI).

Background Knowledge: BMI classification uses World Health Organization guidelines where BMI greater than or equal to 25.0 kg/m^2 indicates overweight status and BMI less than 25.0 kg/m^2 indicates non overweight status. However, BMI reflects complex interactions between multiple factors and shows substantial individual variation. 
BMI varies widely within all demographic categories such as gender, race/ethnicity, education levels and marital status groups. 
Socioeconomic factors influence BMI through food access and activity opportunities. The poverty income ratio (PIR) reflects economic constraints that may affect dietary choices and physical activity access, though individuals make diverse choices within their circumstances.
Physical activity is also a factor that affects the BMI. The physical activity data was measured with an ActiGraph AM-7164 accelerometer and it has some limitations. First, as the device only captures uniaxial movement, activities that primarily involve upper body movement may not have been recorded accurately. Second, it cannot capture water activities because it is not waterproof. Also, device compliance and wear-time varies among individuals. 
Cardiovascular and metabolic markers including systolic/diastolic blood pressure, total cholesterol, HDL, LDL, triglycerides, glucose, and C-reactive protein interact with BMI in complex ways. Some individuals maintain normal weight despite minimal recorded activity due to dietary habits, genetic metabolic differences, unmeasured activities, medical conditions, or body composition variations not captured in this dataset.
BMI prediction requires considering multiple interacting variables while recognizing substantial individual variation that cannot be fully explained by measured factors alone.

Your Task: Based on all information given, while accounting for measurement limitations and individual variations of each person in the list given, please give a prediction about each person's BMI. You need to first give your reasoning process and then give the BMI value. The format of your answer is JSON. Please do not give any additional output, and refer to the following format to give your answer:

Health Profile Data: {input}

```json
{{
“seqn”: int, 
"inference process 1": string, // Please give your inference process of predicting the BMI.
"BMI": int,
"inference process 2": string, // Please give your inference process of predicting the overweight status where 0 is non-overweight and 1 is overweight.
"overweight status": int,
}}
```
Based on the information above, your answer for the entire dataset is:
""" 
    return prompt_template.format(input=health_profile_text)

In [ ]:
# apply the prompt WITHOUT BASELINE WITHOUT COV  

client = openai.OpenAI(api_key='')

if __name__ == "__main__":
    converter = Health_data_text_converter()
    df = pd.read_csv("003_llm_data/data_wide_no_cov_no_bmi.csv", index_col=0)
    llm_results = converter.batch_process(df, '003_llm_data/data_wide_no_cov_no_bmi_converted.csv')

    text_df = pd.read_csv('003_llm_data/data_wide_no_cov_no_bmi_converted.csv')

    checkpoint_file = '003_llm_data/llm_bmi_gpt4o_mini_checkpoint.json'
    results = []
    
    # Load existing progress if available
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            results = json.load(f)
        processed_count = len(results)
        remaining_df = text_df.iloc[processed_count:].reset_index(drop=True)
        print(f"Resuming from row {processed_count}")
    else:
        results = []
        remaining_df = text_df
        processed_count = 0
    
    # Process remaining data
    for index, row in tqdm(remaining_df.iterrows(), total=len(remaining_df), initial=processed_count):
        health_profile = row['text_profile']
        original_seqn = row['seqn']
        full_prompt = create_prompt(health_profile)
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": full_prompt}],
                temperature=0.3,
                max_tokens=4096
            )
            
            # Get response text
            response_text = response.choices[0].message.content
            
            # Extract JSON from ```json blocks
            json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                prediction_data = json.loads(json_str)
                prediction_data['seqn'] = original_seqn
                results.append(prediction_data)
            else:
                # Fallback: try to find any JSON-like structure
                json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
                if json_match:
                    json_str = json_match.group(0)
                    prediction_data = json.loads(json_str)
                    prediction_data['seqn'] = original_seqn
                    results.append(prediction_data)
                else:
                    print(f"No JSON found in response for row {index}")
                    results.append(None)
                    
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            results.append(None)
        
        # Save checkpoint every 10 rows
        if len(results) % 10 == 0:
            with open(checkpoint_file, 'w') as f:
                json.dump(results, f, indent=2)
    
    # Final save
    os.makedirs('003_llm_data', exist_ok=True)
    with open('003_llm_data/llm_bmi_gpt4o_mini.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    # Clean up checkpoint file
    if os.path.exists(checkpoint_file):1
    os.remove(checkpoint_file)
    
    print(f"Saved {len([r for r in results if r is not None])} successful predictions")

In [ ]:
# apply the prompt WITHOUT BASELINE WITH COV  

client = openai.OpenAI(api_key='')

if __name__ == "__main__":
    converter = Health_data_text_converter_cov()
    df = pd.read_csv("003_llm_data/data_wide_cov_no_bmi.csv", index_col=0)
    llm_results = converter.batch_process(df, '003_llm_data/data_wide_cov_no_bmi_converted.csv')

    text_df = pd.read_csv('003_llm_data/data_wide_cov_no_bmi_converted.csv')

    checkpoint_file = '003_llm_data/llm_bmi_gpt4o_mini_cov_checkpoint.json'
    results = []
    
    # Load existing progress if available
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            results = json.load(f)
        processed_count = len(results)
        remaining_df = text_df.iloc[processed_count:].reset_index(drop=True)
        print(f"Resuming from row {processed_count}")
    else:
        results = []
        remaining_df = text_df
        processed_count = 0
    
    # Process remaining data
    for index, row in tqdm(remaining_df.iterrows(), total=len(remaining_df), initial=processed_count):
        health_profile = row['text_profile']
        original_seqn = row['seqn']
        full_prompt = create_prompt(health_profile)
        
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": full_prompt}],
                temperature=0.3,
                max_tokens=4096
            )
            
            # Get response text
            response_text = response.choices[0].message.content
            
            # Extract JSON from ```json blocks
            json_match = re.search(r'```json\s*(.*?)\s*```', response_text, re.DOTALL)
            if json_match:
                json_str = json_match.group(1)
                prediction_data = json.loads(json_str)
                prediction_data['seqn'] = original_seqn
                results.append(prediction_data)
            else:
                # Fallback: try to find any JSON-like structure
                json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
                if json_match:
                    json_str = json_match.group(0)
                    prediction_data = json.loads(json_str)
                    prediction_data['seqn'] = original_seqn
                    results.append(prediction_data)
                else:
                    print(f"No JSON found in response for row {index}")
                    results.append(None)
                    
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            results.append(None)
        
        # Save checkpoint every 10 rows
        if len(results) % 10 == 0:
            with open(checkpoint_file, 'w') as f:
                json.dump(results, f, indent=2)
    
    # Final save
    os.makedirs('003_llm_data', exist_ok=True)
    with open('003_llm_data/llm_bmi_gpt4o_mini_cov.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    # Clean up checkpoint file
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
    
    print(f"Saved {len([r for r in results if r is not None])} successful predictions")